<a href="https://colab.research.google.com/github/Mohamed-El10/Mohamed-El10/blob/main/AnalysisWithLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files


uploaded_train = files.upload()
uploaded_test = files.upload()

Saving Train1.xlsx to Train1.xlsx


Saving Test1.xlsx to Test1.xlsx


In [6]:
import numpy as np # for array, linear algebra
import pandas as pd # for data processing
import matplotlib.pyplot as plt # to crreate stat and visualize data
import matplotlib as mpl  # to visualize data
from sklearn.feature_extraction.text import TfidfVectorizer # converts a collection of raw documents into a matrix

# For Machine Learning Model
from sklearn.linear_model import LogisticRegression # learning algorithm Logistic Regression
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report # for classification report
import nltk
nltk.download('punkt') # Used for sentence tokenizer
nltk.download('stopwords')
nltk.download('wordnet')
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# train Data
trainData = pd.read_excel("Train1.xlsx")

# test Data
testData = pd.read_excel("Test1.xlsx")

In [8]:
#pretraitement traindata
processed_lines = []
for line in trainData['Content']:

    tokens = word_tokenize(line.lower())

    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]

    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in words]

    cleaned_line = ' '.join(lemmatized_tokens)

    # Append the cleaned line to the list
    processed_lines.append(cleaned_line)

trainData['Content'] =processed_lines

In [9]:
#pretraitement testdata
processed_line = []
for line in testData['Content']:

    tokens = word_tokenize(line.lower())

    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]

    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]

    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in words]

    cleaned_line = ' '.join(lemmatized_tokens)

    # Append the cleaned line to the list
    processed_line.append(cleaned_line)

testData['Content'] =processed_line

In [6]:
trainData.head()

,Label,Content
0,pos,zero effect get it title from the main charact...
1,pos,you ve seen this moment before recently a part...
2,pos,you ve probably heard the one about the priest...
3,pos,you ve heard all the hype you ve seen all thei...
4,pos,you ve got to love disney no matter what they ...


dts= pd.DataFrame()
dt= pd.DataFrame()
# Fit TF-IDF vectorizer
tfidf = TfidfVectorizer(min_df=10)

# Transform training and test data using fitted vectorizer
train_tfidf = tfidf.fit_transform(trainData['Content'])
test_tfidf = tfidf.transform(testData['Content'])

# Convert TF-IDF matrices to DataFrames
train_tfidf_df = pd.DataFrame(train_tfidf.toarray(), columns=tfidf.get_feature_names_out())
test_tfidf_df = pd.DataFrame(test_tfidf.toarray(), columns=tfidf.get_feature_names_out())

# Update dt and dts with TF-IDF features
dt = pd.concat([dt, train_tfidf_df], axis=1)
dts = pd.concat([dts, test_tfidf_df], axis=1)

In [10]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)
train_vectors = vectorizer.fit_transform(trainData['Content'])
test_vectors = vectorizer.transform(testData['Content'])

AttributeError: 'csr_matrix' object has no attribute 'shap'

In [14]:
import time

# train the model

logreg = LogisticRegression(random_state=42)

t0 = time.time()
logreg.fit(train_vectors, trainData['Label'])
t1 = time.time()
prediction = logreg.predict(test_vectors)
t2 = time.time()
time_train = t1-t0
time_predict = t2-t1
# results
print("\n\nTraining time: %fs; Prediction time: %fs" % (time_train, time_predict))
report = classification_report(testData['Label'], prediction)
confusion_mat = confusion_matrix(testData['Label'], prediction)

print("\n\nConfusion Matrix is :")
print(confusion_mat)
print("\nClassification Report is :\n",report)




Training time: 0.241505s; Prediction time: 0.000837s


Confusion Matrix is :
[[173  13]
 [ 14 260]]

Classification Report is :
               precision    recall  f1-score   support

         neg       0.93      0.93      0.93       186
         pos       0.95      0.95      0.95       274

    accuracy                           0.94       460
   macro avg       0.94      0.94      0.94       460
weighted avg       0.94      0.94      0.94       460



In [16]:

review = """Do not purchase this product. My cell phone blast when I switched the charger"""

# transform each document into a vector data
review_vector = vectorizer.transform([review]) # vectorizing
print(logreg.predict(review_vector))

['neg']
